In [ ]:

 !pip install transformers datasets evaluate rouge_score accelerate

In [ ]:
 import transformers

 transformers.__version__

'4.44.2'

In [ ]:
 from huggingface_hub import notebook_login

 notebook_login()

In [ ]:
 from huggingface_hub import list_datasets

 datasets_list = list_datasets()

 num = 0
 for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

 num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
DatasetInfo(id='thusinh1969/dummy_instruct_64k', author='thusinh1969', sha='05141d8069a3ecb70e625fab3e1c10b47d2459e5', created_at=datetime.datetime(2024, 9, 20, 10, 10, 34, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 9, 20, 10, 10, 39, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=0, downloads_all_time=None, likes=0, paperswithcode_id=None, tags=['size_categories:n<1K', 'format:parquet', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us'], card_data=None, siblings=None)
DatasetInfo(id='thusinh1969/dummy_instruct_32k', author='thusinh1969', sha='2f9039030efc254b105e5c836bb8e7c5edfa565f', created_at=datetime.datetime(2024, 9, 20, 10, 10, 44, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 9, 20, 10, 10, 48, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=0, downloads_all_time=No

In [ ]:
 from datasets import load_dataset

 ds = load_dataset("navjordj/SNL_summarization")
 ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 10874
    })
    validation: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 819
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 1300
    })
})

In [ ]:
 snl_summarization_ds = load_dataset("navjordj/SNL_summarization", split="train[:1%]") # percentage symbol needs to be followed by the integer without any space in between
 snl_summarization_ds

Dataset({
    features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
    num_rows: 109
})

In [ ]:

 snl_summarization_ds.shape

(109, 7)

In [ ]:

 print(snl_summarization_ds.description)

In [ ]:

 snl_summarization_ds.features

{'id': Value(dtype='int64', id=None),
 'url': Value(dtype='string', id=None),
 'date_scraped': Value(dtype='string', id=None),
 'headline': Value(dtype='string', id=None),
 'category': Value(dtype='string', id=None),
 'ingress': Value(dtype='string', id=None),
 'article': Value(dtype='string', id=None)}

In [ ]:

 snl_summarization_ds.homepage

''

In [ ]:

 snl_summarization_ds[0]

{'id': 5186,
 'url': 'https://snl.no/Gerd_Albrecht',
 'date_scraped': '2023-01-31',
 'headline': 'Gerd Albrecht',
 'category': 'Store norske leksikon,Kunst og estetikk,Musikk,Klassisk musikk,Internasjonale dirigenter',
 'ingress': 'Gerd Albrecht var en tysk dirigent, en av de mest profilerte i sin generasjon. Han huskes blant annet fra ledende stillinger ved Deutsche Oper i Berlin og den statlige operaen i Hamburg.',
 'article': 'Albrecht hadde et stort engasjement for sjeldent oppført repertoar, særlig fra romantikken, og for samtidskomponister. Han var aktiv som gjestedirigent ved mange av ledende orkestrene i Europa og gjestet Oslo-Filharmonien ved flere anledninger. Bakgrunn Albrecht var født og oppvokst i Essen. Faren var musikkviteren Hans Albrecht. Albrecht studerte først ved høyskolen i Kiel og deretter i Hamburg, hvor han hadde dirgenten Wilhelm Brückner-Rüggeberg som lærer. Karriere Albrecht vant førsteprisen ved den internasjonale konkurransen for unge dirigenter i Besançon 

In [ ]:

 snl_summarization_ds=snl_summarization_ds.train_test_split(test_size=0.2, seed=42)
 snl_summarization_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 87
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 22
    })
})

In [ ]:
 def clean_txt(example):
    # Check if the key exists before processing it
    if "article" in example:
        example["article"]  = example["article"].lower()
        example["article"]  = example["article"].replace("/", "")
        example["article"]  = example["article"].replace("\n", "")
        example["article"]  = example["article"].replace("``", "")
        example["article"]  = example["article"].replace('"', '')
        example["article"]  = example["article"].replace("--", "")
    if "highlights" in example:
        example["highlights"]  = example["highlights"].lower()
        example["highlights"]  = example["highlights"].replace("/", "")
        example["highlights"]  = example["highlights"].replace("\n", "")
        example["highlights"]  = example["highlights"].replace("``", "")
        example["highlights"]  = example["highlights"].replace('"', '')
        example["highlights"]  = example["highlights"].replace("--", "")

    return example

In [ ]:
 cleaned_snl_summarization_ds = snl_summarization_ds.map(clean_txt)
 cleaned_snl_summarization_ds

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 87
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 22
    })
})

In [ ]:

 cleaned_snl_summarization_ds['train']["article"][7]

'en dansk bearbeidelse ved sille beyer (1850) fikk tittelen livet i skoven, som gikk inn i herman wildenveys tittel på hans oversettelse for johanne dybwad og nationaltheatret i 1912, as you like it, eller livet i skogen. henrik ibsen iscenesatte stykket i bergen i 1855, hans eneste direkte befatning med shakespeare i teateret. les mer i store norske leksikon william shakespeare storbritannias litteratur'

In [ ]:
 snl_summarization_ds['train']["article"][0]

"Historie Ferden rundt Jorden med Breitling Orbiter III ble gjennomført i perioden 1.–21. mars 1999, med sveitseren Bertrand Piccard (f. 1958), som også fløy det første soldrevne flyet rund Jorden i 2015, og briten Brian Jones (f. 1947) om bord. Ballongen tok av fra Château-d'Oex i Sveits og landet i det sørøstlige Egypt. Under ferden ble det satt flere verdensrekorder for varmluftsballong; korteste tid Jorden rundt (15 dager 10 timer 24 minutter), lengste distanse (40 814 kilometer), lengste varighet (19 dager 21 timer 47 minutter) og største høyde (11 737 meter). Les mer i Store norske leksikon Varmluftballong Solar Impulse 2"

In [ ]:
 cleaned_snl_summarization_ds['train']["article"][0]

"historie ferden rundt jorden med breitling orbiter iii ble gjennomført i perioden 1.–21. mars 1999, med sveitseren bertrand piccard (f. 1958), som også fløy det første soldrevne flyet rund jorden i 2015, og briten brian jones (f. 1947) om bord. ballongen tok av fra château-d'oex i sveits og landet i det sørøstlige egypt. under ferden ble det satt flere verdensrekorder for varmluftsballong; korteste tid jorden rundt (15 dager 10 timer 24 minutter), lengste distanse (40 814 kilometer), lengste varighet (19 dager 21 timer 47 minutter) og største høyde (11 737 meter). les mer i store norske leksikon varmluftballong solar impulse 2"

In [ ]:
 EXAMPLE_TEXT_INDEX = 20

 example_text = cleaned_snl_summarization_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

 example_text

'haag-overenskomsten av 1907 (5) art. 19 har anerkjent angarieretten i et spesielt forhold (i tilknytning til jernbanemateriell), men retten er ikke begrenset bare til slikt materiell. det kan hevdes at bare det generelle prinsippet om militær nødvendighet begrenser angarieretten, det vil si at nøytral eiendom kan beslaglegges etter angarieretten bare dersom dette anses som militært nødvendig.beslagleggelse av nøytrale skip under henvisning til angarieretten fant sted under både første og andre verdenskrig, særlig i storbritannia og usa. det har vært hevdet at angarieretten også gir adgang til å beslaglegge nøytrale skip med last når de er oppbrakt i åpen sjø og ført inn til visitasjon, men dette er neppe alminnelig anerkjent. les mer i store norske leksikon folkerett'

In [ ]:
 model_name= "navjordj/snl-summarization"

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = model_name) # Changed MODEL_NAME to model_name

summary_txt = summarizer(prefix + example_text)

summary_txt

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': 'Angarieretten er folkerettens regler om beslagleggelse av nøytrale eiendom'}]

In [ ]:
summarizer = pipeline("summarization", model = model_name, truncation=True) # Changed MODEL_NAME to model_name


summary_txt = summarizer(prefix + example_text)

summary_txt

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'summary_text': 'Angarieretten er folkerettens regler om beslagleggelse av nøytrale eiendom'}]

In [ ]:
ref_txt = cleaned_snl_summarization_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

ref_txt

'haag-overenskomsten av 1907 (5) art. 19 har anerkjent angarieretten i et spesielt forhold (i tilknytning til jernbanemateriell), men retten er ikke begrenset bare til slikt materiell. det kan hevdes at bare det generelle prinsippet om militær nødvendighet begrenser angarieretten, det vil si at nøytral eiendom kan beslaglegges etter angarieretten bare dersom dette anses som militært nødvendig.beslagleggelse av nøytrale skip under henvisning til angarieretten fant sted under både første og andre verdenskrig, særlig i storbritannia og usa. det har vært hevdet at angarieretten også gir adgang til å beslaglegge nøytrale skip med last når de er oppbrakt i åpen sjø og ført inn til visitasjon, men dette er neppe alminnelig anerkjent. les mer i store norske leksikon folkerett'

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"


In [ ]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

In [ ]:
results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

In [ ]:
summary_txt[0]["summary_text"]

'Angarieretten er folkerettens regler om beslagleggelse av nøytrale eiendom'

In [ ]:

result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.11347517730496454,
 'rouge2': 0.04316546762589929,
 'rougeL': 0.09929078014184396,
 'rougeLsum': 0.09929078014184396}

In [ ]:

article_texts = cleaned_snl_summarization_ds["test"]["article"]

article_summaries = cleaned_snl_summarization_ds["test"]["category"]

In [ ]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 22/22 [03:06<00:00,  8.49s/it]


In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg


{'rouge1': 0.07705002726741858,
 'rouge2': 0.01176096630642085,
 'rougeL': 0.07135683157422289,
 'rougeLsum': 0.0732643684817598}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.33333333333333337,
  0.0,
  0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.1904761904761905,
  0.09999999999999999,
  0.14285714285714288,
  0.09999999999999999,
  0.16666666666666666,
  0.16,
  0.0,
  0.0,
  0.0,
  0.0,
  0.07692307692307693,
  0.1111111111111111,
  0.11111111111111112,
  0.14285714285714288,
  0.0,
  0.0],
 'rouge2': [0.1818181818181818,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.07692307692307691,
  0.0,
  0.0],
 'rougeL': [0.33333333333333337,
  0.0,
  0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.09523809523809525,
  0.09999999999999999,
  0.14285714285714288,
  0.09999999999999999,
  0.16666666666666666,
  0.16,
  0.0,
  0.0,
  0.0,
  0.0,
  0.07692307692307693,
  0.1111111111111111,
  0.11111111111111112,
  0.14285714285714288,
  0.0,
  0.0],
 'rougeLsum': [0.33333333333333337,
  0.0,
  0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.09523809523809525,
  0.099999999

In [ ]:

import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(0, 1)

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,"Altaposten er en lokalavis i Alta kommune, Tro...","Store norske leksikon,Samfunn,Medier,Aviser,No..."
1,José Caboto var en spansk kartograf og kartogr...,"Store norske leksikon,Geografi,Oppdagelsesreiser"
2,Andrzej dmowski var en polsk politiker. Han va...,"Store norske leksikon,Historie,Verdens histori..."
3,i boston ulykken i 1957 var et passasjerskip s...,"Store norske leksikon,Teknologi og industri,Tr..."
4,Allmueskolen var en skoleform der undervisning...,"Store norske leksikon,Samfunn,Utdanning og for..."


In [ ]:

 print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(20, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(20, "Reference_summaries"))

Actual Summary
Angarieretten er folkerettens regler om beslagleggelse av nøytrale eiendom
Reference Summary
Store norske leksikon,Jus,Internasjonal rett,Krigens folkerett


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(18, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(18, "Reference_summaries"))


Actual Summary
Digitale kretser er kretser som utfører logiske operasjoner og
Reference Summary
Store norske leksikon,Teknologi og industri,Elektronikk,Mikroelektronikk


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "bert-base-uncased" # Add this line to define the model name
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

tokenizer(["four-time defending champion"])

{'input_ids': [[101, 2176, 1011, 2051, 6984, 3410, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:

tokens = tokenizer.tokenize("four-time defending champion")

tokens


['four', '-', 'time', 'defending', 'champion']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[2176, 1011, 2051, 6984, 3410]

In [ ]:
from transformers import AutoModelForSequenceClassification, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load a model suitable for BERT's architecture
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 512, truncation = True)

    labels = tokenizer(text_target = examples["headline"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_snl_summarization_ds = cleaned_snl_summarization_ds.map(preprocess_function, batched = True)

tokenized_snl_summarization_ds # Corrected the typo by replacing 'tokenized_snl_summarziation_ds' with 'tokenized_snl_summarization_ds'

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 87
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [ ]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "snl-summarization",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
    # The arguments to truncate overlength sequences are passed during training
    # Remove these arguments from Seq2SeqTrainingArguments
    # max_source_length = 512,
    # max_target_length = 512,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_snl_summarization_ds["train"],
    eval_dataset = tokenized_snl_summarization_ds["train"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


ValueError: Expected input batch_size (16) to match target batch_size (704).

In [ ]:
training_args=Seq2SeqTrainingArguments(
    output_dir="snl_summarization",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    ##max_target_length=512,
)
trainer= Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_snl_summarization_ds["train"],
    eval_dataset=tokenized_snl_summarization_ds["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()